In [24]:
import sys
import os
sys.path.append('..')

from pinecone.grpc import PineconeGRPC as Pinecone
from app.core.config import settings


pc = Pinecone(api_key=settings.pinecone_api_key)

# To get the unique host for an index, 
# see https://docs.pinecone.io/guides/manage-data/target-an-index
index = pc.Index(host="https://clt-tako-rag-kueduco.svc.aped-4627-b74a.pinecone.io")

# To iterate over all result pages using a generator function
all_ids = []
for ids in index.list(prefix='test.pdf', namespace=''):
    all_ids.append(ids)

len(all_ids)

0

In [23]:
index.delete(ids=[item for sublist in all_ids for item in sublist], namespace='')